In [ ]:
# modify datajson
import torch
import os 
import json 
datajson_path_makeup = '/media/pc/hengda1t/hengda/datasets/MT-Dataset/images/mirror_makeup/mirror_makeup_dataset.json'
datajson_path_non_makeup = '/media/pc/hengda1t/hengda/datasets/MT-Dataset/images/mirror_non_makeup/mirror_non_makeup_dataset.json'

with open(datajson_path_non_makeup,'r') as f:
    datajson_non_makeup = json.load(f)
with open(datajson_path_makeup,'r') as f:
    datajson_makeup = json.load(f)

print(len(datajson_non_makeup['labels']))
for x in datajson_makeup['labels']:
    datajson_non_makeup['labels'].append(x)
with open ('/media/pc/hengda1t/hengda/datasets/MT-Dataset/images/mirror_non_makeup/all_mt_dataset.json','w') as f:
   json.dump(datajson_non_makeup,f)
print(len(datajson_non_makeup['labels']))

In [12]:
# landmarks
import dlib
import numpy as np
def get_landmark(filepath, predictor):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)
    for k, d in enumerate(dets):
        shape = predictor(img, d)

    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    return lm
model_path = '../pretrained_models/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(
    model_path)

input_dir_type = 'makeup'
input_img_path = f'/media/pc/hengda1t/hengda/datasets/MT-Dataset/images/{input_dir_type}'
files = os.listdir(input_img_path)
save_dir_path = f'/media/pc/hengda1t/hengda/datasets/MT-Dataset/lms_crop/{input_dir_type}'
os.makedirs(save_dir_path, exist_ok=True)
for file in files:
    if '.png' in file:
        try:
            source_img_path = os.path.join(input_img_path,file)
            lm = get_landmark(source_img_path, predictor)
            np.save(os.path.join(save_dir_path,file.replace('.png','.npy')), lm)
        except:
            print('error_file is :',file)


error_file is : vRX522.png
error_file is : vRX429.png
error_file is : vRX430_mirror.png
error_file is : vRX376_mirror.png
error_file is : vRX336.png
error_file is : vHX457.png
error_file is : vHX457_mirror.png
error_file is : fd4f8721154ec707edd399272e67b9ba.png


In [ ]:
# landmarks
import dlib
import numpy as np
def get_landmark(filepath, predictor):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)
    for k, d in enumerate(dets):
        shape = predictor(img, d)

    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    return lm
model_path = '../pretrained_models/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(
    model_path)

input_dir_type = 'makeup'
input_img_path = f'/media/pc/hengda1t/hengda/datasets/MT-Dataset/images/{input_dir_type}'
files = os.listdir(input_img_path)
save_dir_path = f'/media/pc/hengda1t/hengda/datasets/MT-Dataset/lms_crop/{input_dir_type}'
os.makedirs(save_dir_path, exist_ok=True)
for file in files:
    if 'png' in file:
        source_img_path = os.path.join(input_img_path,file)
        lm = get_landmark(source_img_path, predictor)
        np.save(os.path.join(save_dir_path,file.replace('.png','.npy')), lm)

In [5]:
# preprocess mask
import os
import torch
import numpy as np 
from torchvision import transforms
from torchvision.transforms import functional
from PIL import Image
path_mask = '/media/pc/hengda1t/hengda/EleGANt-eg3d/EleGANt/data_for_test/mask/11220_parsing.png'
lip_class = [12,13]
eyebrow_class =[2,3]
face_class = [1,10]
eye_class = [4,5]
def load_mask(path):
    mask = np.array(Image.open(path).convert('L'))
    mask = torch.FloatTensor(mask).unsqueeze(0)
    mask = functional.resize(mask, 512, transforms.InterpolationMode.NEAREST)
    return mask
def mask_process( mask: torch.tensor):
    '''
    mask: (1, h, w)
    '''        
    mask_lip = (mask == lip_class[0]).float() + (mask == lip_class[1]).float()
    mask_face = (mask == face_class[0]).float() + (mask == face_class[1]).float()

    #mask_eyebrow_left = (mask == eyebrow_class[0]).float()
    #mask_eyebrow_right = (mask == eyebrow_class[1]).float()
    mask_face += (mask == eyebrow_class[0]).float()
    mask_face += (mask == eyebrow_class[1]).float()

    mask_eye_left = (mask == eye_class[0]).float()
    mask_eye_right = (mask == eye_class[1]).float()

    #mask_list = [mask_lip, mask_face, mask_eyebrow_left, mask_eyebrow_right, mask_eye_left, mask_eye_right]
    mask_list = [mask_lip, mask_face, mask_eye_left, mask_eye_right]
    mask_aug = torch.cat(mask_list, 0) # (c, h, w)
    return mask_aug      

mask = load_mask(path_mask)
out = mask_process(mask)
print(mask.shape)


torch.Size([1, 512, 512])


In [ ]:
#encoder result latents  .pt-> .npy
import torch 
import numpy as np 
path = '/media/pc/hengda1t/hengda/e4e_eg3d/save_encoder/inversions/latents/'
dest_path = '/media/pc/hengda1t/hengda/EG3D-projector/eg3d/projector_out/encoder'
files = os.listdir(path)
for file in files:
    files_path = os.path.join(path,file)
    tmp = torch.load(files_path)
    tmp_npy = tmp.numpy()
    np.save(os.path.join(dest_path,file.replace('.pt','.npy')),tmp_npy)

In [ ]:

import os
import numpy as np
import json
root = '/media/pc/hengda1t/hengda/datasets/test_cele'
dataset_json = os.path.join(root,'dataset.json')
root_pngs = os.listdir(root)
with open(dataset_json,'r') as f :
    dataset = json.load(f)
    dataset_labels = dict(dataset['labels'])

for file in root_pngs:
    if '.png' in file:
        camera = dataset_labels[file]      
        np.save(os.path.join(root,file.replace('.png','.npy')),np.array(camera,np.float32))


    print('is over')

import torch

In [ ]:
# get latent code by optimization

import os
path_test_imgs = '/media/pc/hengda1t/hengda/EleGANt-eg3d/EleGANt/data_for_test/test_mt_optimize/'
dirs = ['epoch_89','epoch_93','epoch_94','epoch_100', 'epoch_106', 'epoch_112']
for di in dirs:
    
    files = os.listdir(os.path.join(path_test_imgs,di))
    print(len(files))
    for file in files:
        if '.png' in file:
            path_file = os.path.join(path_test_imgs,di,file)
            # cmd = f'CUDA_VISIBLE_DEVICES=0 python ./run_projector.py --image_path {path_file} --datajson /media/pc/hengda1t/hengda/datasets/MT-Dataset/images/mirror_non_makeup/all_mt_dataset.json --latent_space_type w_plus --outdir=../results/optimize_out --network ../pretrained_models/ffhq512-128.pkl'
            # os.system(cmd)       
